In [29]:
import requests
import jwt
import time
import secrets
import json
from jwcrypto import jwk
baseUrl = "https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/fhir"
session = requests.Session()
headers={"Content-Type": "application/json"}
alg = "RS384"
client_id = "nist"

config = session.get(baseUrl + "/.well-known/smart-configuration")
print(config.text)
config.json.__get__("authorization_endpoint")
tokenUrl = config.json()["token_endpoint"]
scope = config.json()["scopes_supported"][0]
print(tokenUrl, scope)


# https://ri-connectathon.open-iis.org/ImmMessagingServer/client_Service

{"token_endpoint_auth_signing_alg_values_supported":["RS384","ES384"],"grant_types_supported":["client_credentials"],"token_endpoint_auth_methods_supported":["private_key_jwt"],"scopes_supported":["system/*.rc"],"token_endpoint":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/auth/token"}
https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/auth/token system/*.rc


In [31]:
with open("RS384.key", "rb") as pemfile: 
    key = jwk.JWK.from_pem(pemfile.read())
with open("RS384.key", "rb") as keyfile: 
    secretKey = keyfile.read()
with open("RS384.key.pub", "rb") as keyfilepub: 
    publicKey = keyfilepub.read()
jwt_claims = {"iss": client_id, "sub": client_id, "aud": tokenUrl, "exp": int(time.time()) + 300, "jti": secrets.token_urlsafe()}
jwt_headers = {"alg":alg, "kid":key.get("kid"), "typ":"JWT"}
encoded_jwt = jwt.encode(payload=jwt_claims, key=secretKey, algorithm=alg, headers=jwt_headers)
decode = jwt.decode(encoded_jwt, publicKey, algorithms=[alg],audience=tokenUrl)
login = session.post(tokenUrl, params={"client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer", "client_assertion": encoded_jwt,  "grant_type": "client_credentials", "scope": scope},headers={"content-type": "application/x-www-form-urlencoded"})
# print(login, login.content)
if login.status_code == 200:
    token = json.loads(login.text)["access_token"]
    print(token)

eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJGSElSIEltbXVuaXphdGlvbnMiLCJleHAiOjE3MDU1MjQ2MzQsImlhdCI6MTcwNTUyNDMzNCwianRpIjoiZGIzODA5YjItY2E0MC00OTJjLThkMTMtMDJkMzI3Njc3YTYwIiwidXNlcm5hbWUiOiJqYm9uem8iLCJwYXRpZW50SWQiOm51bGwsInByb3ZpZGVySWQiOm51bGwsInBvdGVudGlhbFBjcCI6bnVsbCwic2VydmVyVXJsIjoiaHR0cHM6Ly9yaS1jb25uZWN0YXRob24ub3Blbi1paXMub3JnL2ZoaXIta2lkc25ldC1peiIsInVzZXJUeXBlIjpudWxsLCJ0YXJnZXQiOm51bGwsInJldHVyblRvIjoiS05TRUFSQ0giLCJzZWNvbmRhcnlSZXR1cm5UbyI6IlJJQ0FJUl9JWiIsImF1eCI6bnVsbCwiaW50ZXJuYWxVc2VyIjpmYWxzZSwiaW50ZXJuYWxVc2VyT3B0aW9ucyI6eyJsb2FkRXhpc3RpbmdJbW11bml6YXRpb25zIjp0cnVlLCJpbW11bml6YXRpb25JbnB1dFJvd3MiOjV9LCJkaXNwbGF5Ijp7InBhdGllbnRHZW5kZXJJc1Zpc2libGUiOmZhbHNlLCJoZWxwIjp7InVybCI6IiIsInRleHQiOiJIZWxwIn19fQ.4EHUDUmHof-wZbADKMpxyc7hBE3ygs-CVaiHDtQDTSMbHiwOVVDMyF5b29gbKwxl8c_EMNG9UCizF_arhQATKg


In [24]:
result = session.get(f"{baseUrl}/Group/1102357/$export", headers={"Authorization": f"Bearer {token}", "prefer": "respond-async"})
print(result)
print(result.content)
print(result.headers)

<Response [202]>
b''
{'Date': 'Wed, 17 Jan 2024 20:23:42 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'X-Powered-By': 'HAPI FHIR 6.10.0 REST Server (FHIR Server; FHIR 4.0.1/R4)', 'Content-Location': 'https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/fhir/$export-poll-status?jobId=9c0d0f04-881f-46e0-9409-1cd2eaac0efc', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


In [26]:
checkurl=  result.headers["Content-Location"]
check = session.get(checkurl, headers={"Authorization": f"Bearer {token}"})
print(check)
print(check.content)
print(check.headers)

<Response [200]>
b'{"transactionTime":"2024-01-17T20:23:42.734Z","request":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/fhir/Group/1102357/$export","requiresAccessToken":true,"output":[{"type":"Immunization","url":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/export/output/9c0d0f04-881f-46e0-9409-1cd2eaac0efc/Immunization_1.ndjson","count":100},{"type":"ImmunizationEvaluation","url":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/export/output/9c0d0f04-881f-46e0-9409-1cd2eaac0efc/ImmunizationEvaluation_1.ndjson","count":100},{"type":"ImmunizationRecommendation","url":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/export/output/9c0d0f04-881f-46e0-9409-1cd2eaac0efc/ImmunizationRecommendation_1.ndjson","count":1600},{"type":"Patient","url":"https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/export/output/9c0d0f04-881f-46e0-9409-1cd2eaac0efc/Patient_1.ndjson","count":100}],"error":[]}'
{'Date': 'Wed, 17 Jan 2024 20:25:16 GMT', 'Server': 'Apach

In [27]:
ndj = "https://ri-connectathon.open-iis.org/fhir-kidsnet-iz/v1/export/output/9c0d0f04-881f-46e0-9409-1cd2eaac0efc/Immunization_1.ndjson"
check = session.get(ndj, headers={"Authorization": f"Bearer {token}"})
print(check)
print(check.content)
print(check.headers)

<Response [200]>
b'{"resourceType":"Immunization","id":"11158358","meta":{"lastUpdated":"2023-01-14T06:08:22.000-04:00"},"extension":[{"url":"https://health.ri.gov/immunization-source-of-update","valueString":"02"},{"url":"https://health.ri.gov/vaccine-category-group","extension":[{"url":"https://health.ri.gov/vaccine-category-group#name","valueString":"COVID-19"},{"url":"https://health.ri.gov/vaccine-category-group#sequence","valueInteger":30}]}],"identifier":[{"use":"usual","value":"11158358"},{"use":"official","value":"7343242.212.2021-01-19T000000"}],"status":"completed","vaccineCode":{"coding":[{"system":"http://hl7.org/fhir/sid/cvx","code":"212","display":"COVID-19 PF JSN"}],"text":"COVID-19, vector-nr, rS-Ad26, PF, 0.5 mL (JSN)"},"patient":{"reference":"7343242","type":"Patient"},"occurrenceDateTime":"2021-01-19T00:00:00-04:00","recorded":"2023-01-14T06:08:22-04:00","primarySource":false,"manufacturer":{"reference":"PFR","type":"Organization","display":"Pfizer, Inc"},"site":{"co